In [16]:
import numpy
from scipy.ndimage.interpolation import shift

def TR(inputs: dict):
    '''
    向量运算
    '''
    high = inputs['high']
    low = inputs['low']
    close = inputs['close']
    preClose = shift(close, 1, cval=np.nan)
    
    return np.amax(np.array([abs(high-low), abs(high-preClose), abs(preClose-low)]), axis=0)

def ATR(tr: numpy.ndarray, n):
    '''
    增量算法
    '''
    atr = np.full_like(tr, fill_value=np.nan)
    set_first_atr = False
    for i in range(len(tr)):
        if not set_first_atr:
            if not np.isnan(tr[i]):
                atr[i] = tr[i]
                set_first_atr = True
        else:
            atr[i] = ((n-1)/n)*atr[i-1] + tr[i]/n    
            
    return atr

def DM(inputs: dict):
    '''
    向量运算
    '''
    high = inputs['high']

    preHigh = shift(high, 1, cval=np.nan)
    low = inputs['low']
    preLow = shift(low, 1, cval=np.nan)

    deltaHigh = high - preHigh
    deltaLow = preLow - low

    DMplus = np.full_like(deltaHigh, fill_value=np.nan)
    DMminus = np.full_like(deltaHigh, fill_value=np.nan)

    condition1= np.logical_or(np.logical_and(deltaHigh < 0, deltaLow < 0), deltaHigh == deltaLow)
    DMplus[condition1] = 0
    DMminus[condition1] = 0
    condition2 = np.logical_and(deltaHigh > deltaLow, np.isnan(DMplus))
    DMplus[condition2] = deltaHigh[condition2]
    DMminus[condition2] = 0
    condition3 = np.logical_and(deltaHigh < deltaLow, np.isnan(DMplus))
    DMplus[condition3] = 0
    DMminus[condition3] = deltaLow[condition3]
    
    return DMplus, DMminus

def ADM(DMplus:numpy.ndarray, DMminus: numpy.ndarray, n) -> (numpy.ndarray, numpy.ndarray):
    '''
    增量算法
    '''
    admPlus = np.full_like(DMplus, fill_value=np.nan)
    admMinus = np.full_like(DMminus, fill_value=np.nan)
    set_first_adm = False
    assert len(DMplus) == len(DMminus)
    for i in range(len(DMplus)):
        if not set_first_adm:
            if not np.isnan(DMplus[i]):
                admPlus[i] = DMplus[i]
                admMinus[i] = DMminus[i]
                set_first_adm = True
        else:
            admPlus[i] = ((n-1)/n)*admPlus[i-1] + DMplus[i]/n    
            admMinus[i] = ((n-1)/n)*admMinus[i-1] + DMminus[i]/n

    return admPlus, admMinus
def DI(admPlus: numpy.ndarray, admMinus: numpy.ndarray, atr: numpy.ndarray) -> (numpy.ndarray, numpy.ndarray):
    '''
    向量运算
    '''
    DIplus = 100 * admPlus/atr
    DIminus = 100 * admMinus/atr
    
    return DIplus, DIminus

def DX_helper(DIplus: numpy.ndarray, DIminus: numpy.ndarray):
    '''
    向量运算
    '''
    return 100*abs(DIplus - DIminus)/(DIplus + DIminus)
def DX(inputs: dict, n: int):
    '''
    API
    '''
    np.set_printoptions(suppress=True)
    tr = TR(inputs)
    atr = ATR(tr, n)
    dmPlus, dmMinus = DM(inputs)
    admPlus, admMinus = ADM(dmPlus, dmMinus, n)
    DIplus, DIminus = DI(admPlus, admMinus, atr)
    
    print('tr: \n', tr)
    print('atr: \n', atr)
    print('dmPlus: \n', dmPlus)
    print('dmMinus: \n', dmMinus)
    print('admPlus: \n', admPlus)
    print('admMinus: \n', admMinus)
    print('DIplus: \n', DIplus)
    print('DIminus: \n', DIminus)

    
    return DX_helper(DIplus, DIminus)
    
def ADX_helper(dx: numpy.ndarray, n):
    '''
    增量算法
    '''
    adx = np.full_like(dx, fill_value=np.nan)
    set_first_adx = False
    for i in range(len(dx)):
        if not set_first_adx:
            if not np.isnan(dx[i]):
                adx[i] = dx[i]
                set_first_adx = True
        else:
            adx[i] = ((n-1)/n)*adx[i-1] + dx[i]/n    
            
    return adx    

def ADX(inputs: dict, n: int):
    '''
    API
    '''
    dx = DX(inputs, n)
    return ADX_helper(dx, n)

In [7]:
import numpy as np

inputs = {
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100)
}

In [17]:
DX(inputs, 14)


tr: 
 [       nan 0.73376423 0.15833086 0.78029306 0.51949382 0.57772689
 0.77196791 0.56609433 0.53573843 0.26690686 0.3174526  0.71711073
 0.86683434 0.65230693 0.83919696 0.23160646 0.64817483 0.70668471
 0.52886342 0.2881654  0.61209729 0.67812468 0.57290147 0.90650974
 0.27533007 0.73509572 0.71583496 0.69477097 0.11435233 0.60078499
 0.72243222 0.33581864 0.49891294 0.36155063 0.73238789 0.40564605
 0.54230459 0.31823417 0.06396664 0.65016763 0.26282845 0.53467924
 0.72202537 0.37171604 0.17900381 0.93448764 0.45951558 0.68513686
 0.53903509 0.36022159 0.44455084 0.32262465 0.59604064 0.89792889
 0.42203717 0.44421965 0.66476902 0.04360869 0.42605565 0.40060825
 0.24160314 0.26465388 0.76714174 0.17068515 0.07400878 0.03784393
 0.47656075 0.46352312 0.67332951 0.89226808 0.29969709 0.75538102
 0.40360003 0.09523996 0.83131512 0.56369794 0.63767652 0.08823602
 0.49403505 0.80295524 0.90245402 0.5106073  0.07159617 0.92393529
 0.27593757 0.56712464 0.19889541 0.68699114 0.4706258  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in less


array([         nan, 100.        ,  42.89762651,  53.18610433,
        28.6923729 ,  33.87721738,  26.45874391,  22.67865324,
        22.67865324,  28.19100556,  14.66868872,  21.11372056,
        21.11372056,   4.01147141,  24.88833185,  24.88833185,
        12.56066171,  12.56066171,  19.35392773,   6.62206077,
         2.13446417,   2.13446417,  17.110068  ,   8.60732317,
         8.60732317,   3.26256284,  19.35049826,  19.35049826,
        14.56276319,  14.56276319,  30.62006233,  30.62006233,
        41.40252197,  26.29842323,  18.65840147,  24.07500886,
        24.62016784,  31.95064459,  34.74121189,  28.93749336,
        17.6196127 ,  17.6196127 ,  30.15647313,  22.99264123,
         6.43624835,   6.43624835,  13.88868582,  11.57113789,
        27.42778891,  27.42778891,   9.36768878,   9.36768878,
        20.13116917,  14.33638696,  14.33638696,   0.92815541,
         0.92815541,  12.17719675,   1.94222661,   4.9176148 ,
         1.41663213,   2.00315291,  12.13502005,  12.13

In [14]:
from talib import abstract

abstract.DX(inputs)

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,  2.22223637,
        2.22223637, 10.63655727, 10.63655727,  3.06757822, 12.29030824,
       18.77275266, 18.77275266, 30.13240081, 21.44316295, 21.44316295,
       15.9250091 , 29.00170832, 29.00170832, 24.1493093 , 24.1493093 ,
       37.67491298, 37.67491298, 46.93870098, 31.85726393, 24.1748348 ,
       29.05003684, 29.54203412, 36.18141361, 38.72056495, 32.91590231,
       21.55319101, 21.55319101, 33.22631537, 26.04758477,  9.3893416 ,
        9.3893416 , 11.19028462,  8.96047021, 25.09039525, 25.09039525,
        7.54350299,  7.54350299, 18.42420574, 12.76101149, 12.76101149,
        2.18061967,  2.18061967, 11.00093323,  0.93327393,  3.92200862,
        0.47350815,  1.06237078, 12.87873452, 12.87873452, 10.91106474,
        9.70567142,  4.78351214,  9.26709665,  6.98251807, 25.58

In [18]:
DX(inputs, 14) - abstract.DX(inputs)

tr: 
 [       nan 0.73376423 0.15833086 0.78029306 0.51949382 0.57772689
 0.77196791 0.56609433 0.53573843 0.26690686 0.3174526  0.71711073
 0.86683434 0.65230693 0.83919696 0.23160646 0.64817483 0.70668471
 0.52886342 0.2881654  0.61209729 0.67812468 0.57290147 0.90650974
 0.27533007 0.73509572 0.71583496 0.69477097 0.11435233 0.60078499
 0.72243222 0.33581864 0.49891294 0.36155063 0.73238789 0.40564605
 0.54230459 0.31823417 0.06396664 0.65016763 0.26282845 0.53467924
 0.72202537 0.37171604 0.17900381 0.93448764 0.45951558 0.68513686
 0.53903509 0.36022159 0.44455084 0.32262465 0.59604064 0.89792889
 0.42203717 0.44421965 0.66476902 0.04360869 0.42605565 0.40060825
 0.24160314 0.26465388 0.76714174 0.17068515 0.07400878 0.03784393
 0.47656075 0.46352312 0.67332951 0.89226808 0.29969709 0.75538102
 0.40360003 0.09523996 0.83131512 0.56369794 0.63767652 0.08823602
 0.49403505 0.80295524 0.90245402 0.5106073  0.07159617 0.92393529
 0.27593757 0.56712464 0.19889541 0.68699114 0.4706258  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in less
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in less


array([         nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,  22.66609548,  22.66609548,
         1.92410444,   1.92410444,  16.2863495 ,  -5.66824747,
       -16.63828849, -16.63828849, -13.02233281, -12.83583979,
       -12.83583979, -12.66244626,  -9.65121006,  -9.65121006,
        -9.5865461 ,  -9.5865461 ,  -7.05485065,  -7.05485065,
        -5.53617902,  -5.5588407 ,  -5.51643334,  -4.97502798,
        -4.92186628,  -4.23076903,  -3.97935307,  -3.97840896,
        -3.93357831,  -3.93357831,  -3.06984224,  -3.05494355,
        -2.95309325,  -2.95309325,   2.69840121,   2.61066768,
         2.33739366,   2.33739366,   1.82418579,   1.82418579,
         1.70696342,   1.57537548,   1.57537548,  -1.25246426,
        -1.25246426,   1.17626352,   1.00895269,   0.99560618,
         0.94312399,   0.94078212,  -0.74371448,  -0.74